# Metadata stats

In [7]:
import pandas as pd
import os
from compare_MT_api_performance import get_mean_attribute_per_occupation
occupation_category_data = pd.read_csv('data/metadata/occupation_categories.tsv', sep='\t')
# convert categories => list
category_vars = list(filter(lambda x: x.startswith('categories_'), occupation_category_data.columns))
occupation_category_data = occupation_category_data.assign(**{
    var_i: occupation_category_data.loc[:, var_i].apply(lambda x: x.split(';') if type(x) is str else [])
    for var_i in category_vars
})
## get other metadata
age_occupation_data = pd.read_csv('data/metadata/age_occupation_data.tsv', sep='\t')
income_gender_occupation_data = pd.read_csv('data/metadata/female_work_representation_data.tsv', sep='\t')
# fix column names
age_occupation_data.rename(columns={'Median\n age': 'median_age', 'Total, 16\nyears and\n over': 'total_pop'},
                           inplace=True)
income_gender_occupation_data.rename(
    columns={'Median earnings': 'median_earnings', 'Percentage of women in occupational group': 'women_pct',
             'Number of full-time workers': 'total_pop'}, inplace=True)
# remove occupation w/ insufficient data
age_occupation_data = age_occupation_data[age_occupation_data.loc[:, 'median_age'] != '–']
# fix numbers
number_vars = ['median_earnings', 'total_pop', 'median_age']
for number_var_i in number_vars:
    if (number_var_i in income_gender_occupation_data.columns):
        income_gender_occupation_data = income_gender_occupation_data.assign(**{
            number_var_i: income_gender_occupation_data.loc[:, number_var_i].apply(
                lambda x: x.replace(',', '') if type(x) is str else x).astype(float)
        })
    if (number_var_i in age_occupation_data.columns):
        age_occupation_data = age_occupation_data.assign(**{
            number_var_i: age_occupation_data.loc[:, number_var_i].apply(
                lambda x: x.replace(',', '') if type(x) is str else x).astype(float)
        })

# income
income_attr = 'median_earnings'
occupation_category_data = occupation_category_data.assign(**{
    'income': occupation_category_data.loc[:, 'categories_BOLS'].apply(
        lambda x: get_mean_attribute_per_occupation(income_attr, x, income_gender_occupation_data))
})
# gender
gender_attr = 'women_pct'
occupation_category_data = occupation_category_data.assign(**{
    'women_pct': occupation_category_data.loc[:, 'categories_BOLS'].apply(
        lambda x: get_mean_attribute_per_occupation(gender_attr, x, income_gender_occupation_data))
})
# age
age_attr = 'median_age'
occupation_category_data = occupation_category_data.assign(**{
    'age': occupation_category_data.loc[:, 'category_DOL'].apply(
        lambda x: get_mean_attribute_per_occupation(age_attr, [x], age_occupation_data))
})
display(occupation_category_data.head())
occupation_category_data.to_csv('data/metadata/combined_occupation_meta_data.tsv', sep='\t', index=False)

,occupation,categories_BOLS,category_DOL,income,women_pct,age
0,analyst,"[Management analysts, Budget analysts, Credit ...",Budget analysts;Computer systems analysts;Cred...,84776.087072,41.089307,NaN
1,artist,[Artists and related workers],Artists and related workers,49032.000000,38.200000,43.3
2,athlete,[Athletes and sports competitors],NaN,NaN,NaN,NaN
3,author,[Writers and authors],Writers and authors,61189.000000,55.500000,44.8
4,baker,[Bakers],Bakers,29241.000000,57.400000,41.7
